In [ ]:
import warnings, os
import datetime as dt
from dateutil.relativedelta import relativedelta as rtd
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin([*SYMBOLS]) & Raw['tf'].isin([*TFRAMES])] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['H1'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    WEEK_DAY = { 'Sunday':1, 'Monday':2, 'Tuesday':3, 'Wednesday':4, 'Thursday':5, 'Friday':6, 'Saturday':7 }

    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    Src['Week.Day'] = Query['datetime'].dt.day_name().map(WEEK_DAY)
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['Hilo']     = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']        = Calc_by['Open'].transform('first')
    Calc['ID High']         = Calc_by['High'].expanding().max()     .reset_index(drop=1)
    Calc['ID Low']          = Calc_by['Low'].expanding().min()      .reset_index(drop=1)

    Calc['ID Change']       = Calc['Close']   - Calc['Day Open']
    Calc['ID Hilo']         = Calc['ID High'] - Calc['ID Low']

    Calc['ID Sign']         = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Calc['ID Sign Bull']    = Calc['ID Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['ID Sign Bear']    = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['ID Chg Abs']      = Calc['ID Change'].abs()
    Calc['ID Chg Pos']      = Calc['ID Chg Abs'] * Calc['ID Sign Bull'] 
    Calc['ID Chg Neg']      = Calc['ID Chg Abs'] * Calc['ID Sign Bear'] 

    Calc['ID HL Pos']       = Calc['ID Hilo'] * Calc['ID Sign Bull']
    Calc['ID HL Neg']       = Calc['ID Hilo'] * Calc['ID Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [15]:
def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 

        Df = Calc[Calc['Time'] <= dt.time(HR,00)]

        pipe.append({
            'Before': dt.time(HR+1,00), 
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)

In [16]:
STATS(Calc, Col='ID Change')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1240,-7,479,-2355,-290,10,280,2045
1,11:00:00,2480,-1,634,-2640,-375,5,375,4690
2,12:00:00,3720,-9,739,-3630,-455,-5,430,4690
3,13:00:00,4960,-10,812,-3690,-510,-5,475,4690
4,14:00:00,6205,-11,862,-3690,-545,-5,510,4690
5,15:00:00,7450,-11,907,-3835,-575,0,550,5555
6,16:00:00,8695,-13,948,-4520,-600,0,580,5555
7,17:00:00,9940,-13,986,-4520,-621,0,615,5555
8,18:00:00,11185,-15,1023,-4520,-645,5,630,5555
9,19:00:00,11867,-14,1041,-5150,-655,5,645,5555


In [17]:
STATS(Calc, Col='ID Chg Abs')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1240,364,312,0,135,290,515,2355
1,11:00:00,2480,479,415,0,170,375,685,4690
2,12:00:00,3720,559,484,0,200,445,790,4690
3,13:00:00,4960,619,526,0,220,490,880,4690
4,14:00:00,6205,662,553,0,240,530,945,4690
5,15:00:00,7450,698,579,0,250,565,1005,5555
6,16:00:00,8695,730,605,0,265,590,1045,5555
7,17:00:00,9940,760,629,0,275,620,1085,5555
8,18:00:00,11185,787,654,0,285,640,1115,5555
9,19:00:00,11867,800,666,0,290,650,1130,5555


In [18]:
STATS(Calc, Col='ID Chg Pos')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,636,348,301,0,115,270,498,2045
1,11:00:00,1250,474,418,0,160,365,690,4690
2,12:00:00,1858,550,480,0,195,430,790,4690
3,13:00:00,2468,612,522,0,210,480,881,4690
4,14:00:00,3097,652,547,0,220,510,950,4690
5,15:00:00,3729,686,572,0,235,550,1005,5555
6,16:00:00,4362,715,593,0,245,580,1045,5555
7,17:00:00,5001,742,615,0,260,605,1080,5555
8,18:00:00,5641,766,638,0,270,625,1110,5555
9,19:00:00,6001,777,648,0,275,635,1125,5555


In [19]:
STATS(Calc, Col='ID Chg Neg')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,604,-381,323,-2355,-530,-300,-150,-5
1,11:00:00,1230,-484,412,-2640,-685,-375,-185,-5
2,12:00:00,1862,-567,487,-3630,-790,-455,-210,-5
3,13:00:00,2492,-627,530,-3690,-875,-505,-235,-5
4,14:00:00,3108,-671,558,-3690,-945,-545,-250,-5
5,15:00:00,3721,-710,586,-3835,-1005,-575,-265,-5
6,16:00:00,4333,-745,617,-4520,-1045,-600,-280,-5
7,17:00:00,4939,-778,643,-4520,-1085,-625,-290,-5
8,18:00:00,5544,-809,669,-4520,-1120,-655,-300,-5
9,19:00:00,5866,-824,683,-5150,-1145,-665,-305,-5


In [20]:
STATS(Calc, Col='ID Hilo')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,1240,745,332,215,505,670,905,2965
1,11:00:00,2480,961,457,215,625,880,1190,5635
2,12:00:00,3720,1113,535,215,735,1025,1370,5950
3,13:00:00,4960,1225,584,215,815,1135,1520,5950
4,14:00:00,6205,1308,613,215,880,1215,1625,5950
5,15:00:00,7450,1380,644,215,935,1285,1700,7400
6,16:00:00,8695,1444,672,215,980,1345,1770,7400
7,17:00:00,9940,1501,701,215,1015,1400,1835,7400
8,18:00:00,11185,1552,726,215,1050,1440,1895,7400
9,19:00:00,11867,1573,739,215,1070,1460,1920,7400


In [21]:
STATS(Calc, Col='ID HL Pos')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,636,736,317,215,500,670,895,2355
1,11:00:00,1250,956,455,215,630,875,1190,5635
2,12:00:00,1858,1104,530,215,735,1020,1365,5950
3,13:00:00,2468,1214,580,215,805,1128,1510,5950
4,14:00:00,3097,1294,605,215,875,1210,1605,5950
5,15:00:00,3729,1364,637,215,930,1275,1680,7400
6,16:00:00,4362,1424,663,215,965,1335,1750,7400
7,17:00:00,5001,1479,690,215,1000,1395,1810,7400
8,18:00:00,5641,1526,714,215,1035,1435,1850,7400
9,19:00:00,6001,1547,726,215,1045,1450,1875,7400


In [22]:
STATS(Calc, Col='ID HL Neg')

,Before,count,mean,std,min,25%,50%,75%,max
0,10:00:00,604,-754,347,-2965,-911,-670,-510,-275
1,11:00:00,1230,-965,459,-3345,-1189,-880,-625,-275
2,12:00:00,1862,-1121,541,-4390,-1370,-1025,-735,-275
3,13:00:00,2492,-1236,588,-4715,-1520,-1145,-824,-275
4,14:00:00,3108,-1322,621,-4800,-1640,-1225,-890,-275
5,15:00:00,3721,-1396,650,-4800,-1720,-1295,-950,-275
6,16:00:00,4333,-1463,680,-5205,-1800,-1355,-995,-275
7,17:00:00,4939,-1524,710,-5225,-1875,-1405,-1025,-275
8,18:00:00,5544,-1579,738,-5225,-1945,-1450,-1070,-275
9,19:00:00,5866,-1600,751,-5720,-1974,-1470,-1085,-275


# Snippets

In [23]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)

    Calc['Day Open']        = Calc_by['Open'].transform('first')
    Calc['ID High']         = Calc_by['High'].expanding().max()     .reset_index(drop=1)
    Calc['ID Low']          = Calc_by['Low'].expanding().min()      .reset_index(drop=1)

    Calc['ID Change']       = Calc['Close']   - Calc['Day Open']
    Calc['ID Hilo']         = Calc['ID High'] - Calc['ID Low']

    Calc['ID Sign']         = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Calc['ID Sign Bull']    = Calc['ID Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Calc['ID Sign Bear']    = Calc['ID Change'].apply(lambda x: -1 if (x <  0) else nan)

    Calc['ID Chg Abs']      = Calc['ID Change'].abs()
    Calc['ID Chg Pos']      = Calc['ID Chg Abs'] * Calc['ID Sign Bull'] 
    Calc['ID Chg Neg']      = Calc['ID Chg Abs'] * Calc['ID Sign Bear'] 

    Calc['ID HL Pos']       = Calc['ID Hilo'] * Calc['ID Sign Bull']
    Calc['ID HL Neg']       = Calc['ID Hilo'] * Calc['ID Sign Bear']
    return Calc


def STATS(Calc, Col):
    pipe = []
    for HR in [9,10,11,12,13,14,15,16,17,18]: 
        Df = Calc[Calc['Time'] <= dt.time(HR,00)]
        pipe.append({
            'Before': dt.time(HR+1,00), 
            **Df[Col].describe().round(0).astype(int)
        })
    pass
    return pd.DataFrame(pipe)